In [1]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 10213466112 (9.51 GB)
Present Capacity: 3239010304 (3.02 GB)
DFS Remaining: 2982391808 (2.78 GB)
DFS Used: 256618496 (244.73 MB)
DFS Used%: 7.92%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 92
	Missing blocks (with replication factor 1): 92
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.27.0.5:9866 (project-4-data4lyf4-dn-2.project-4-data4lyf4_default)
Hostname: 4c44021d23eb
Decommission Status : Normal
Configured Capacity: 10213466112 (9.51 GB)
DFS Used: 256618496 (244.73 MB)
Non DFS Used: 6957678592 (6.48 GB)
DFS Remaining: 2982391808 (2.78 GB)
DFS Used%: 2.51%
DFS Remaini

In [2]:
import time
time.sleep(30)

In [3]:
#q9
import requests
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=LISTSTATUS")
status = r.json()
total_size = status['FileStatuses']['FileStatus'][0]['length']
block_size = status['FileStatuses']['FileStatus'][0]['blockSize']
counts = {'lost':0}
for offset in range(0, total_size, block_size):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(offset))
    if r.status_code == 403:
        counts['lost'] += 1
    else:
        url = r.json()['Location']
        id = url[url.find('http://') + len('http://'):url.find(':9864')]
        if id in counts:
            counts[id] += 1
        else:
            counts[id] = 1
counts

{'lost': 92, '4c44021d23eb': 75}

In [11]:
#q10
import pyarrow as pa
import pyarrow.fs
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

cnt = 0  
for offset in range(0, total_size, block_size):
    r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={}&noredirect=true".format(offset))
    if r.status_code != 403:
        with hdfs.open_input_file("hdfs://boss:9000/single.csv") as f:
            blk = f.read_at(block_size, offset)
            cnt += blk.count(b"Single Family")

cnt

<Response [403]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [403]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [200]>
<Response [403]>
<Response [200]>
<Response [403

0